In [1]:
%load_ext autoreload
%autoreload 2

# add project folder
import os
from os.path import dirname, join
PROJECT_FOLDER = dirname(dirname(os.getcwd()))
os.sys.path.append(PROJECT_FOLDER)

# add dependencies
from underthesea.utils.col_analyzer import UDAnalyzer
from underthesea.utils.col_script import RawToUDDataset

# init folder
DATASETS_FOLDER = join(PROJECT_FOLDER, "datasets")
COL_FOLDER = join(DATASETS_FOLDER, "UD_Vietnamese-COL")
raw_file = join(COL_FOLDER, "corpus", "raw", "202108.txt")

Load dictionary with 31327 words.


In [2]:
%%capture
raw_file = join(COL_FOLDER, "corpus", "raw", "202108.txt")
dataset = RawToUDDataset(raw_file)

ud_file = join(COL_FOLDER, "corpus", "ud", "202108.txt")
dataset.write(ud_file)

In [3]:
analyzer = UDAnalyzer()

In [4]:
analyzer.analyze_dataset_len(dataset)

Number of sentences 50


In [5]:
analyzer.analyze_sent_ids(dataset)

In [6]:
analyzer.analyze_words(dataset)

Words
[(',', 86), ('.', 41), ('của', 13), ('và', 13), ('trong', 13), ('là', 12), ('đã', 12), ('được', 11), ('ngày', 11), ('người', 10), ('để', 10), ('ở', 9), ('với', 9), ('hà nội', 9), ('từ', 9), ('bán', 9), ('mua', 9), ('đặt', 8), ('tháng', 8), ('mới', 8), ('này', 8), ('ra', 8), ('cho', 8), ('không', 8), ('2', 7), ('vào', 7), ('có', 7), ('các', 7), ('về', 7), ('nhà', 7), ('cá', 7), ('nướng', 7), ('nhiều', 6), ('chợ', 6), ('chị', 6), ('rất', 5), ('đó', 5), ('những', 5), ('nhanh', 5), ('covid-19', 5), ('nghiện', 5), ('(', 5), (')', 5), ('con', 5), ('vàng', 5), ('vàng mã', 5), ('đây', 4), ('năm', 4), ('hai', 4), ('nhưng', 4)]
Out of dictionary
Corpus words:  573
OOV words
{'1.178', 'ma túy', '2.709', '7/8', 'kéo dài', 'long biên', 'đtk', 'group', 'ba đình', 'elo', 'quang liêm', 'yên nghĩa', 'lionel messi', 'phó tổng thống', '13/8', '2021', '150.000', 'nha trang', 'tajikistan', '2.128', 'the guardian', 'hasibullah stanikzai', 'saleh', 'việt nam', 'tùy', '"', '4', 'điện biên', '12-8', 'bn'